In [12]:
import copy
from pprint import pprint

import numpy as np
import yaml
from easydict import EasyDict
from tqdm import tqdm

import wandb

In [13]:
api = wandb.Api()

In [27]:
with open("../configs/base_config.yaml", "r") as file:
    base_config = EasyDict(yaml.safe_load(file))

with open("../configs/igs_config.yaml", "r") as file:
    igs_config = EasyDict(yaml.safe_load(file))

with open("../configs/relative_flatness_config.yaml", "r") as file:
    relative_flatness_config = EasyDict(yaml.safe_load(file))

In [28]:
pprint(base_config)

{'batch_size': 256,
 'cifar_dir': '../data/cifar',
 'config_dir': '../configs',
 'learning_rate': 0.05,
 'metrics': ['acc', 'ece', 'squared_euclidean_norm', 'fisher_rao_norm'],
 'models_dir': '../models',
 'momentum': 0.9,
 'num_epochs': 50,
 'seed': [43, 91, 17],
 'sharpness_batch_size': 25000,
 'sharpness_dataset_size': -1,
 'vgg_config': 'vgg_config.json',
 'wandb': {'entity': 'r252_project',
           'experiment_name': '',
           'project': 'VGG19_CIFAR10_FINAL'}}


In [29]:
models = ["best_model", "model", "temp_best_model", "temp_model"]

In [30]:
metrics = ["acc", "ece", "squared_euclidean_norm", "relative_flatness", "IGS"]

In [43]:
approaches = ["no-regularisation", "augmentation", "dropout_0.5", "weight_decay0.0005"]

In [44]:
runs = api.runs(f"{base_config.wandb.entity}/{base_config.wandb.project}")

In [48]:
results = {model: {approach: {metric: {} for metric in metrics} for approach in approaches} for model in models}

for run in tqdm(runs):
    for approach in approaches:
        if run.name.startswith(f"VGG19-CIFAR10-baseline-with-{approach}"):
            seed = run.name.split("-seed-")[-1]
            for metric in metrics:
                if metric in run.config['metrics']:
                    for model in models:

                        results[model][approach][metric][seed] = run.summary[f"eval_metrics_{model}/{metric}"]

100%|██████████| 31/31 [00:00<00:00, 40093.56it/s]


In [49]:
pprint(results)

{'best_model': {'augmentation': {'IGS': {'91': 'NaN'},
                                 'acc': {'17': 0.879,
                                         '43': 0.8954,
                                         '91': 0.8935},
                                 'ece': {'17': 0.07430009543895721,
                                         '43': 0.06152530014514923,
                                         '91': 0.06169174239039421},
                                 'relative_flatness': {'17': 14.161212314094882,
                                                       '43': 10.83865855152908,
                                                       '91': 11.235430257707776},
                                 'squared_euclidean_norm': {'17': 53.53863951003177,
                                                            '43': 49.74028485739116,
                                                            '91': 51.0907264777731}},
                'dropout_0.5': {'IGS': {'43': 0.005866386504041858},
       

In [52]:
processed_results = copy.deepcopy(results)

for model in models:
    for approach in approaches:
        for metric in metrics:
            num_results = len(results[model][approach][metric])
            if num_results == 1:
                if list(results[model][approach][metric].values())[0] == 'NaN':
                    processed_results[model][approach][metric] = (0, 0)
                else:
                    processed_results[model][approach][metric] = (list(results[model][approach][metric].values())[0], 0)
            else:
                processed_results[model][approach][metric] = (np.mean(list(results[model][approach][metric].values())),
                                                    np.std(list(results[model][approach][metric].values())))

In [59]:
prefix = r"""\begin{table}
    \centering
    \begin{tabular}{l>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}}
    \toprule
    \textbf{Regularizer} & \textbf{Error (\%)} & \textbf{ECE} & \textbf{Squared euclidean norm} & \textbf{Relative flatness} & \textbf{IGS} \\ \midrule"""

for model in models:

    table = prefix

    for approach in approaches:
        approach_name = approach
        approach_name = approach_name.replace("_", " ").replace("-", " ")
        approach_name_italicized_latex = f"\\textit{{{approach_name}}}"
        table += f"\n\t{approach_name_italicized_latex} "
        for metric in metrics:
            mean = processed_results[model][approach][metric][0]
            std = processed_results[model][approach][metric][1]
            if metric == "acc":
                mean = 100 - mean * 100
                std *= 100
            elif metric == "ece":
                mean *= 100
                std *= 100
            table += f"& {mean:.2f}$_{{\pm {std:.2f}}}$ "
        table += r"\\"

    model = model.replace("_", " ")

    table += r"""
    \bottomrule \\
    \end{tabular}
    \caption{Results for VGG-19 on CIFAR-10 for """ + model + r"""}
    \label{tab:measures}
\end{table}
    """

    print(table)
    print("\n")

\begin{table}
    \centering
    \begin{tabular}{l>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}>{\raggedleft\arraybackslash}p{2cm}}
    \toprule
    \textbf{Regularizer} & \textbf{Error (\%)} & \textbf{ECE} & \textbf{Squared euclidean norm} & \textbf{Relative flatness} & \textbf{IGS} \\ \midrule
	\textit{no regularisation} & 14.44$_{\pm 0.05}$ & 0.11$_{\pm 0.00}$ & 49.32$_{\pm 4.12}$ & 2.22$_{\pm 1.00}$ & 0.00$_{\pm 0.00}$ \\
	\textit{augmentation} & 11.07$_{\pm 0.73}$ & 0.07$_{\pm 0.01}$ & 51.46$_{\pm 1.57}$ & 12.08$_{\pm 1.48}$ & 0.00$_{\pm 0.00}$ \\
	\textit{dropout 0.5} & 13.87$_{\pm 0.19}$ & 0.11$_{\pm 0.00}$ & 39.13$_{\pm 0.31}$ & 1.20$_{\pm 0.21}$ & 0.01$_{\pm 0.00}$ \\
	\textit{weight decay0.0005} & 18.22$_{\pm 1.26}$ & 0.11$_{\pm 0.01}$ & 33.21$_{\pm 0.93}$ & 4.48$_{\pm 0.22}$ & 0.00$_{\pm 0.00}$ \\
    \bottomrule \\
    \end{tabular}
    \caption{